In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import statsmodels.api as sm
from datetime import timedelta, datetime

In [2]:
directory = 'C:\\Users\\Emma Hegermiller\\Git\\price-prediction\\data'


df_import =pd.read_csv(directory + '\\sample.csv')

C:\Users\Emma Hegermiller\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Drop invalid row that are missing vin or list_price
# Drop variables that are not vehicle attributes
# Drop variables that cannot be used in price predication at the time of listing
drop_cols = ['index_pandas', 
             'days_on_market', 
             'listing_date_end', 'shift_buyer_region']
df_valid = df_import.dropna(subset=['vin', 'list_price']).drop(columns = drop_cols).drop_duplicates().astype({'year': 'str'})
print("Columns: n/{}".format(df_valid.columns))
print("Row count: {}".format(len(df_valid)))
df_valid.sort_values(["vin", "listing_date_begin"]).head(100)

Columns: n/Index(['vin', 'listing_date_begin', 'year', 'make', 'model', 'trim',
       'body_style', 'transmission', 'mileage', 'accident_count',
       'fuel_economy_city', 'fuel_economy_highway', 'exterior_color', 'msrp',
       'is_cpo', 'seller_city', 'seller_state', 'seller_type',
       'seller_is_franchise_dealer', 'seller_is_online_only',
       'seller_ships_nationwide', 'list_price'],
      dtype='object')
Row count: 395450


,vin,listing_date_begin,year,make,model,trim,body_style,transmission,mileage,accident_count,...,exterior_color,msrp,is_cpo,seller_city,seller_state,seller_type,seller_is_franchise_dealer,seller_is_online_only,seller_ships_nationwide,list_price
57683,0HGCV3F93LA014514,2021-02-23,2020,Honda,Accord,NaN,Sedan,Continuously Variable Transmission,7318.0,NaN,...,Blue,NaN,NaN,"Arlington, TX",TX,DEALER,True,NaN,NaN,26595.0
8404,19XFC1F30LE000679,2021-03-28,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,24772.0,0.0,...,Molten Lava Pearl,NaN,True,"North Richland Hills, TX",TX,DEALER,True,NaN,NaN,21795.0
4298,19XFC1F30LE001184,2021-01-11,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,21846.0,0.0,...,Platinum White Pearl,NaN,True,"Albuquerque, NM",NM,DEALER,True,NaN,NaN,20959.0
4299,19XFC1F30LE001184,2021-01-11,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,21846.0,0.0,...,Platinum White Pearl,NaN,True,"Albuquerque, NM",NM,DEALER,True,NaN,NaN,21505.0
6629,19XFC1F30LE001699,2021-04-17,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,26203.0,0.0,...,Silver,NaN,NaN,"Newport News, VA",VA,DEALER,True,NaN,NaN,21495.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23833,19XFC1F31LE010203,2020-08-24,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,1750.0,0.0,...,Cosmic Blue Metallic,24900.0,NaN,"Fort Myers, FL",FL,DEALER,True,NaN,NaN,21990.0
610,19XFC1F31LE010511,2020-10-31,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,13047.0,0.0,...,Platinum White Pearl,NaN,NaN,"Grapevine, TX",TX,DEALER,True,NaN,NaN,20000.0
6,19XFC1F31LE010752,2021-04-04,2020,Honda,Civic,EX Sedan FWD,Sedan,Automatic,18971.0,0.0,...,Silver,NaN,NaN,"Houston, TX",TX,DEALER,NaN,NaN,NaN,22998.0
8,19XFC1F31LE010752,2021-04-04,2020,Honda,Civic,EX Sedan FWD,Sedan,Automatic,18916.0,0.0,...,Silver,NaN,NaN,"San Antonio, TX",TX,DEALER,NaN,NaN,NaN,22998.0


In [ ]:
# Split into train and test
unique_vins = df_valid['vin'].unique()
print("Number of unique vins: {}".format(len(unique_vins)))

train_vin = np.random.choice(unique_vins, size=int(len(unique_vins)*0.8), replace=False, p=None)
test_vin = unique_vins[~np.in1d(unique_vins, train_vin)]

df_train = df_valid[df_valid['vin'].isin(train_vin)]
df_test = df_valid[df_valid['vin'].isin(test_vin)]
print("test row percentage: {}%".format(round(100*(len(df_test)/len(df_valid)))))

# Check takes long to run
# vin_duplicated = []
# for i in df_test.index:
#     if df_test.loc[i, 'vin'] in list(df_train['vin']):
#         vin_duplicated.append(df_test.loc[i, 'vin'])

# assert len(vin_duplicated)==0, "Vin number in test and train"

Number of unique vins: 283993


In [ ]:
# Export data 
df_train.to_csv(directory + '\\train_initial.csv')
df_test.to_csv(directory + '\\test_initial.csv')